In [16]:
import json
from quam_components.components import *

from pathlib import Path
root_folder = Path("./output")
root_folder.mkdir(exist_ok=True)

In [17]:
num_qubits = 3

quam = QuAM()

for idx in range(num_qubits):
    # Create qubit components
    local_oscillator_qubit = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_qubit)

    mixer_qubit = Mixer(
        id=f"mixer_q{idx}",
        local_oscillator=local_oscillator_qubit,
        port_I=1,
        port_Q=2,
        frequency_drive=5e9,
    )

    transmon = Transmon(
        id=idx,
        xy=XYChannel(
            mixer=mixer_qubit, pi_amp=10e-3, pi_length=40, anharmonicity=200e6
        ),
        z=ZChannel(port=5),
    )
    quam.qubits.append(transmon)

    # Create resonator components
    local_oscillator_resonator = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_resonator)

    resonator_mixer = Mixer(
        id=f"mixer_r{idx}",
        local_oscillator=local_oscillator_resonator,
        port_I=3,
        port_Q=4,
        frequency_drive=5e9,
    )

    readout_resonator = ReadoutResonator(id=idx, mixer=resonator_mixer)
    quam.resonators.append(readout_resonator)

In [18]:
quam.save(root_folder / "quam.json")

In [20]:
qua_config = quam.build_config()
json.dump(qua_config, open(root_folder / "qua_config.json", "w"), indent=2)